# Results comparison between Trajectron++ and our method
Comparing the results of our method with the Trajectron++ paper's method. The goal of the comparison is to evaluate Trajectron++ on a different dataset than it was trained on to allow more fair comparison.

In [1]:
import sys
import os
import dill
import json
import argparse
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv

## Helper functions

In [2]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

In [3]:
def process_data(input_data):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (7 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

## Method evaluation logic

In [26]:
def evaluate_our_method(data, params, dataset_title='', single_output = False):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, _ = generative_model.predict(x_data, y_data, params, trajectory_length=12)

                    best_fde = None
                    best_ade = None
                    
                    if single_output:
                        avg_x = np.mean(all_pred_x, axis=0)
                        avg_y = np.mean(all_pred_y, axis=0)
                        best_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                        best_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                    else:
                        for i in range(len(all_pred_x)):
                            current_pred_x = all_pred_x[i]
                            current_pred_y = all_pred_y[i]

                            fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_fde == None or fde < best_fde:
                                best_fde = fde

                            ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                            if best_ade == None or ade < best_ade:
                                best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

In [27]:
def evaluate_cvm_with_scenarios(data, dataset_title='', history_length=8):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y = [], []
                    
                    # CVM
                    for i in range(20):
                        history_x = x_data[-history_length:]
                        history_y = y_data[-history_length:]
                        assert len(history_x) == history_length
                        
                        if i == 0:
                            vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                            vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]
                        else:
                            vel_x = [(history_x[i] - history_x[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_x))]
                            vel_y = [(history_y[i] - history_y[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_y))]
                        
                        assert len(vel_x) == history_length-1
                        avg_vel_x = np.mean(vel_x)
                        avg_vel_y = np.mean(vel_y)
                        
                        pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                        pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                        assert len(pred_x) == 12
                        
                        all_pred_x.append(pred_x)
                        all_pred_y.append(pred_y)
                    

                    best_fde = None
                    best_ade = None
                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

def evaluate_cvm(data, dataset_title='', history_length=8):
    tot = 0
    all_fde = []
    all_ade = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    history_x = x_data[-history_length:]
                    history_y = y_data[-history_length:]
                    assert len(history_x) == history_length

                    vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                    vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]

                    assert len(vel_x) == history_length-1
                    avg_vel_x = np.mean(vel_x)
                    avg_vel_y = np.mean(vel_y)

                    pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                    pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                    assert len(pred_x) == 12

                    fde = calculate_FDE(pred_x, pred_y, x_gt, y_gt)
                    ade = calculate_ADE(pred_x, pred_y, x_gt, y_gt)

                    all_fde.append(fde)
                    all_ade.append(ade)
                    
    return all_fde, all_ade


## Automated results comparison - Best of 20

In [28]:
def read_trajectron_data(trajectron_resultset_name, suffix='best_of'):
    trajectron_fde = []
    with open('pedestrians/results/' + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open('pedestrians/results/' + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [40]:
def evaluate_all_datasets(our_method_params, files, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=False):
    base_path = './pedestrians/raw/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_ar3_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_long_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_short_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for file_idx, file in enumerate(files):
        data = pd.read_csv(base_path + file, sep='\t', index_col=False, header=None)
        data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

        data = process_data(data)

        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[file_idx])
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[file_idx], suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))
        
        ## Trajectron with interactions
        trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_ar3_resultset_names[file_idx])
        trajectron_ar3_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_ar3_fde))
        trajectron_ar3_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ar3_ade))
        
        if evaluate_most_likely:
            trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_resultset_names[file_idx], suffix='most_likely')
            trajectron_ar3_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_ar3_fde))
            trajectron_ar3_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ar3_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        num_predictable_trajectories = get_total_predictable_slices(data)
        assert len(trajectron_fde) == num_predictable_trajectories
        assert len(trajectron_ade) == num_predictable_trajectories

        ## Ours
        our_fde_best_of_20, our_ade_best_of_20 = evaluate_our_method(data, our_method_params[file_idx], dataset_title=trajectron_resultset_names[file_idx])
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))
        
        if evaluate_most_likely:
            our_fde_single, our_ade_single = evaluate_our_method(data, our_method_params[file_idx], dataset_title=trajectron_resultset_names[file_idx], single_output=True)
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_single))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_single))

        ## CVM
        cvm_fde_best_of, cvm_ade_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[file_idx])
        cvm_fde_short_history_best_of, cvm_ade_short_history_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[file_idx], history_length=2)
        cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
        cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
        cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
        cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))
        
        if evaluate_most_likely:
            cvm_fde, cvm_ade = evaluate_cvm(data, dataset_title=trajectron_resultset_names[file_idx])
            cvm_fde_short_history, cvm_ade_short_history = evaluate_cvm(data, dataset_title=trajectron_resultset_names[file_idx], history_length=2)
            cvm_long_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde))
            cvm_long_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade))
            cvm_short_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_short_history))
            cvm_short_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_short_history))
    
    # Omit univ for now as it takes very long to evaluate
    
    ## Univ is a separate case as it has 2 scenes
    filename1 = './pedestrians/raw/univ/test/students001.txt'
    filename2 = './pedestrians/raw/univ/test/students003.txt'
    univ_data_1 = pd.read_csv(filename1, sep='\t', index_col=False, header=None)
    univ_data_2 = pd.read_csv(filename2, sep='\t', index_col=False, header=None)
    univ_data_1.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']
    univ_data_2.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

    univ_data_1 = process_data(univ_data_1)
    univ_data_2 = process_data(univ_data_2)

    ## Ours
    our_fde_best_of_1, our_ade_best_of_1 = evaluate_our_method(univ_data_1, our_method_params[-1], dataset_title='univ 1')
    our_fde_best_of_2, our_ade_best_of_2 = evaluate_our_method(univ_data_2, our_method_params[-1], dataset_title='univ 2')

    our_fde_best_of_20 = our_fde_best_of_1 + our_fde_best_of_2
    our_ade_best_of_20 = our_ade_best_of_1 + our_ade_best_of_2

    ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
    ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))
    
    if evaluate_most_likely:
        our_fde_ml_1, our_ade_ml_1 = evaluate_our_method(univ_data_1, our_method_params[-1], dataset_title='univ 1', single_output=True)
        our_fde_ml_2, our_ade_ml_2 = evaluate_our_method(univ_data_2, our_method_params[-1], dataset_title='univ 2', single_output=True)

        our_fde_ml = our_fde_ml_1 + our_fde_ml_2
        our_ade_ml = our_ade_ml_1 + our_ade_ml_2

        ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_ml))
        ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_ml))

    ## CVM
    
    cvm_fde_best_of_1, cvm_ade_best_of_1 = evaluate_cvm_with_scenarios(univ_data_1, dataset_title='univ 1')
    cvm_fde_best_of_2, cvm_ade_best_of_2 = evaluate_cvm_with_scenarios(univ_data_2, dataset_title='univ 2')

    cvm_fde_best_of = cvm_fde_best_of_1 + cvm_fde_best_of_2
    cvm_ade_best_of = cvm_ade_best_of_1 + cvm_ade_best_of_2

    cvm_fde_short_hist_best_of_1, cvm_ade_short_hist_best_of_1 = evaluate_cvm_with_scenarios(univ_data_1, dataset_title='univ 1', history_length=2)
    cvm_fde_short_hist_best_of_2, cvm_ade_short_hist_best_of_2 = evaluate_cvm_with_scenarios(univ_data_2, dataset_title='univ 2', history_length=2)

    cvm_fde_short_history_best_of = cvm_fde_short_hist_best_of_1 + cvm_fde_short_hist_best_of_2
    cvm_ade_short_history_best_of = cvm_ade_short_hist_best_of_1 + cvm_ade_short_hist_best_of_2

    cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
    cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
    cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
    cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))
    
    if evaluate_most_likely:
        cvm_fde_ml_1, cvm_ade_ml_1 = evaluate_cvm(univ_data_1, dataset_title='univ 1')
        cvm_fde_ml_2, cvm_ade_ml_2 = evaluate_cvm(univ_data_2, dataset_title='univ 2')

        cvm_fde_ml = cvm_fde_ml_1 + cvm_fde_ml_2
        cvm_ade_ml = cvm_ade_ml_1 + cvm_ade_ml_2

        cvm_fde_short_hist_ml_1, cvm_ade_short_hist_ml_1 = evaluate_cvm(univ_data_1, dataset_title='univ 1', history_length=2)
        cvm_fde_short_hist_ml_2, cvm_ade_short_hist_ml_2 = evaluate_cvm(univ_data_2, dataset_title='univ 2', history_length=2)

        cvm_fde_short_history_ml = cvm_fde_short_hist_ml_1 + cvm_ade_short_hist_ml_1
        cvm_ade_short_history_ml = cvm_ade_short_hist_ml_1 + cvm_ade_short_hist_ml_2

        cvm_long_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_ml))
        cvm_long_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_ml))
        cvm_short_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_short_history_ml))
        cvm_short_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_short_history_ml))  
        
    ## Trajectron
    trajectron_fde, trajectron_ade = read_trajectron_data('univ_vel')
    trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
    trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
    
    if evaluate_most_likely:
        trajectron_fde, trajectron_ade = read_trajectron_data('univ_vel', suffix='most_likely')
        trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))
    
    ## Trajectron with interactions
    trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data('univ_ar3')
    trajectron_ar3_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_ar3_fde))
    trajectron_ar3_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ar3_ade))
    
    if evaluate_most_likely:
        trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data('univ_ar3', suffix='most_likely')
        trajectron_ar3_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_ar3_fde))
        trajectron_ar3_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ar3_ade))
    
    
    return [
        ours_results,
        trajectron_results,
        trajectron_ar3_results,
        cvm_long_results,
        cvm_short_results
    ]

In [ ]:
params_eth = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.15,

    "ANGLE_CHANGE_PROB": 0.2,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.35, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

params_rest = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.9,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.75,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.1,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.05,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.40, 0.65, 0.85, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

our_method_params = [params_eth, params_eth, params_rest, params_rest]

files = [
    'eth/test/biwi_eth.txt', 
    'hotel/test/biwi_hotel.txt', 
    'zara1/test/crowds_zara01.txt', 
    'zara2/test/crowds_zara02.txt'
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel', 
    'zara1_vel', 
    'zara2_vel'
]

trajectron_ar3_resultset_names = [
    'eth_ar3', 
    'hotel_ar3', 
    'zara1_ar3', 
    'zara2_ar3'
]

res = evaluate_all_datasets(our_method_params, files, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)

Ours - univ 1:   7%|██████▍                                                                                        | 30/444 [02:51<44:24,  6.44s/it]

In [37]:
ours_results = res[0]
trajectron_results = res[1]
trajectron_ar3_results = res[2]
cvm_long_results = res[3]
cvm_short_results = res[4]

In [39]:
index = [
    'ETH', 
    'Hotel', 
    'Zara 1', 
    'Zara 2',
    'Univ'
]

df_data_best_of_20_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)

df_data_best_of_20_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)

df_data_most_likely_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
    'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['FDE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
}

df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)

df_data_most_likely_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
    'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['ADE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
}

df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)

ValueError: Length of values (5) does not match length of index (4)

In [ ]:
#df_fde
df_data_best_of_20_fde.style.highlight_min(color = 'lightgreen', axis = 1)

In [ ]:
df_best_of_20_ade.style.highlight_min(color = 'lightgreen', axis = 1)

In [ ]:
df_most_likely_fde.style.highlight_min(color = 'lightgreen', axis = 1)

In [ ]:
df_most_likely_ade.style.highlight_min(color = 'lightgreen', axis = 1)

## Trying many sets of params

In [ ]:
params1 = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.15,

    "ANGLE_CHANGE_PROB": 0.2,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.35, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

params2 = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.9,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.75,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.1,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.05,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.40, 0.65, 0.85, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

params3 = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.8,

    "STOP_PROB": 0.1,

    "DISCOUNT_AVG_PROB": 0.3,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.2,
    "VELOCITY_CHANGE_NOISE": 0.15,

    "ANGLE_CHANGE_PROB": 0.2,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.35, 0.68, 0.95, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 6, 5, 5, 3] # Total 20 traj
}

params4 = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.7,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.5,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.1,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.05,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.4, 0.7, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 7, 7, 5] # Total 20 traj
}

params5 = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 1000,
    "CONST_VEL_MODEL_PROB": 0.7,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.3,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.1,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.025,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.10, 0.4, 0.7, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 7, 7, 5] # Total 20 traj
}

params6 = {
    "NOISE": 0.05,
    "NO_OF_TRAJECTORIES": 2000,
    "CONST_VEL_MODEL_PROB": 0.7,

    "STOP_PROB": 0.05,

    "DISCOUNT_AVG_PROB": 0.3,
    "DISCOUNT_LOWER_BOUND": 0.5,

    "VELOCITY_CHANGE_PROB": 0.1,
    "VELOCITY_CHANGE_NOISE": 0.1,

    "ANGLE_CHANGE_PROB": 0.025,
    "ANGLE_CHANGE_NOISE": 1,

    "GROUP_PERCENTAGES": [0.05, 0.5, 0.8, 1.0],
    "GROUP_CLUSTER_COUNT": [1, 8, 6, 5] # Total 20 traj
}

param_set = [params1, params2, params3, params4, params5, params6]

In [ ]:
all_df_bo20_fde = []
all_df_bo20_ade = []
all_df_ml_fde = []
all_df_ml_ade = []
for idx, params in enumerate(param_set):
    our_method_params = [params, params, params, params]
    res = evaluate_all_datasets(our_method_params, files, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)
    
    ours_results = res[0]
    trajectron_results = res[1]
    trajectron_ar3_results = res[2]
    cvm_long_results = res[3]
    cvm_short_results = res[4]

    index = [
    'ETH', 
    'Hotel', 
    'Zara 1', 
    'Zara 2',
    'Univ'
    ]

    df_data_best_of_20_fde = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
        'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['FDE'], index = index),
        'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
    }

    df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)
    all_df_bo20_fde.append(df_best_of_20_fde)

    df_data_best_of_20_ade = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
        'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['ADE'], index = index),
        'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
    }

    df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)
    all_df_bo20_ade.append(df_best_of_20_ade)

    df_data_most_likely_fde = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
        'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['FDE'], index = index),
        'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
    }

    df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)
    all_df_ml_fde.append(df_most_likely_fde)

    df_data_most_likely_ade = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
        'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['ADE'], index = index),
        'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
    }

    df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)
    all_df_ml_ade.append(df_most_likely_ade)
    
    print(idx)
    print(df_fde)
    print(df_ade)